In [ ]:
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light
import pickle
from pathlib import Path

In [ ]:
pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic, 
    spectrometer, 
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

In [ ]:
# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range

In [9]:
# factor_sigma = 0.509
pa.go_to_ple_target(scan_range_constr[0])
time.sleep(2)
w1 = high_finesse_wavemeter_remote.get_current_wavelength()

pa.go_to_ple_target(scan_range_constr[1])
time.sleep(2)
w2 = high_finesse_wavemeter_remote.get_current_wavelength()

factor_sigma = (w2 - w1) * 1000 / 26 #GHz 

In [10]:
factor_sigma

0.5033879614171173

In [12]:
#prepare
#switch off green and turn on the 405 nm repump
ibeam_smart.disable()

# cobolt.enable_modulated()
# cobolt.set_laser_modulated_power(power = 5)

results = []

power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R"
folder = os.path.join(folder, r"auto_D")
if not os.path.exists(folder):
    os.mkdir(folder) 

for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    results_poi["poi_name"] = poi_name
    results_poi["factor_sigma"] = factor_sigma
    
    # go to a defect
    pa.set_resonant_power(power = 300)
    time.sleep(1)

     #configure slow scanning for the wavemeter scanning optimizations
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    scanner_gui._osd.change_settings({'scan_frequency': {"x": 10, "y": 10, "z": 10},
                                    "scan_resolution": {"x": 15, "y": 15, "z":80},
                                    "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
    scanner_gui._osd.accept()
    time.sleep(0.5)
    pa.go_to_poi(poi_name, opt_times=1)
    

    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.2)

    #Check how the PLE look like
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range",
            poi_name=poi_name, folder_name=folder_defect)
    results_poi.update({
        "ple_scans" : {
        "full_range" : ple_data_logic.last_saved_files_paths 
            }
        })

    if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value or (res["center"].value - 2 * res["sigma"].value < scan_range_constr[0]) or (res["center"].value + 2 * res["sigma"].value > scan_range_constr[1]): 
        print("NO PLE or it is at the rim")
        pa.save_ple(tag = "failed_PLE_scan",
            poi_name=poi_name, folder_name=folder_defect)
        
        continue
    #since there is a PLE -- optimize_all
    pa.go_to_ple_target(res["center"].value)
    #get the center wavelength:
    pa.optimize_ple()
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)

    time.sleep(0.5)

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 50, "y": 50, "z": 50},
                                        "scan_resolution": {"x": 15, "y": 15, "z":80},
                                        "scan_range": {"x": 1e-6, "y": 1e-6, "z": 4e-6}})
    scanner_gui._osd.accept()
    time.sleep(0.5)
    pa.optimize_all()
    
    #Now we now where the defect is
    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
    

    pa.set_resonant_power(power = power_steps[1])
    fine_scan_range = ( 
        res["center"].value - 5 * res["sigma"].value ,
        res["center"].value + 5 * res["sigma"].value
                    )
    time.sleep(1)
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)

    scan_range_sat = 2 * res["sigma"].value

    pa.save_ple(tag = "5sigma_range",
                poi_name=poi_name, folder_name=folder_defect)
    time.sleep(0.5)

    results_poi.update({
        "ple_scans" : {
        "5sigma_range" : ple_data_logic.last_saved_files_paths 
        }
    })


    # now we run saturation with a cw repump
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))
    results_poi["saturation"] = {}
    idx_no_ple = None
    
    # #405 repump
    # cobolt.enable_modulated()
    # cobolt.set_laser_modulated_power(power = 5)
    
    for idx, power in enumerate(power_steps):
        
        pa.set_resonant_power(power = power)
        time.sleep(1)
        fine_scan_range = ( 
        res["center"].value - scan_range_sat ,
        res["center"].value + scan_range_sat
                    )
        res = pa.do_ple_scan(lines = 2, in_range=fine_scan_range)
        time.sleep(1)
        #save_results
        pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=saturation_folder)
        #save_plots
        if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
            idx_no_ple = idx
            break
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value: 
            #ple is gone.
            idx_no_ple = idx
            break

        
        #get the center wavelength:
        
        pa.go_to_ple_target(res["center"].value)
        pa.optimize_ple()
        wavelength = high_finesse_wavemeter_remote.get_current_wavelength()

        # #charge_lifetime:
        # for i in range(10):
        #     pa.optimize_ple()
        #     time.sleep(5)

        results_poi.update({"saturation": 
                            {f"{power}":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "wavelength": wavelength
                                }
                                }})
    if idx_no_ple is None:
        continue
    
    # # photostability study
    # for i in range(idx_no_ple):
    #     idx_no_ple_checked = idx_no_ple - i
    #     pa.set_resonant_power(power = int(power_steps[idx_no_ple_checked])) #take the last but one visible ple power
    #     #reionize with blue
    #     #fine_scan_range = ( 
    #     #res["center"].value - 5 * res["sigma"].value ,
    #     #res["center"].value + 5 * res["sigma"].value
    #     #            )
    #     pa.one_pulse_repump("blue")
    #     time.sleep(0.5)
    #     res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    #     fine_scan_range = ( 
    #     res["center"].value - scan_range_sat,
    #     res["center"].value + scan_range_sat
    #             )
    #     if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
        
    #             continue
    #     if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value: 
    #             #ple is gone.
        
    #             continue
        
    #     pa.save_ple(tag = f"{power}_prepumped_405",
    #         poi_name=poi_name, folder_name=saturation_folder)
    #     time.sleep(1)
    #     results_poi.update({"no_repump": {
    #                     "power": power,
    #                     "scan_data": ple_data_logic.last_saved_files_paths,
    #                     "sigma": res["sigma"].value,
    #                     "sigma_stderr": res["sigma"].stderr
    #                     }}
    #                     )
    #     break
    # #no we now the power where we still see the ple
    #os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))
    

    # cobolt.enable_modulated()
    # cobolt.set_laser_modulated_power(power = 5)
    # pa.set_resonant_power(power = power_steps[idx_no_ple_checked])
    # time.sleep(1)
    # res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)

    # #ple_gui._osd.change_settings({"scan_range": {"a": 5000}}) # 3GHz
    # #ple_gui._osd.accept()

    #  #take the last but one visible ple power
    # #reionize with blue
    # pa.one_pulse_repump("blue")
    # fine_scan_range = ( 
    # res["center"].value - 5 * res["sigma"].value ,
    # res["center"].value + 5 * res["sigma"].value
    #             )
    # time.sleep(1)
    # res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    # time.sleep(1)
    # pa.save_ple(tag = f"no_repump_{power_steps[idx_no_ple_checked]}", folder_name=folder_defect) 


    # #no repump saturation

    # os.mkdir(no_repump_saturation := os.path.join(folder_defect, "no_repump_saturation"))
    # idx_no_ple_ohne_repump = None
    # for idx, power in enumerate(power_steps[idx_no_ple_checked : ]):
    
    #     pa.set_resonant_power(power = power)
    #     time.sleep(1)
    #     fine_scan_range = ( 
    #     res["center"].value - scan_range_sat ,
    #     res["center"].value + scan_range_sat
    #                 )
    #     pa.one_pulse_repump("blue")
    #     time.sleep(0.5)
    #     res = pa.do_ple_scan(lines = 5, in_range=fine_scan_range)
    #     time.sleep(1)
    #     #save_results
    #     pa.save_ple(tag = f"{power}",
    #         poi_name=poi_name, folder_name=no_repump_saturation)
    #     #save_plots
    #     if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
    #         idx_no_ple_ohne_repump = idx
    #         break
    #     if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value: 
    #         #ple is gone.
    #         idx_no_ple_ohne_repump = idx
    #         break

    #     #get the center wavelength:
        
    #     pa.go_to_ple_target(res["center"].value)
    #     pa.optimize_ple()
    #     wavelength = high_finesse_wavemeter_remote.get_current_wavelength()

    #     # #charge_lifetime:
    #     # for i in range(10):
    #     #     pa.optimize_ple()
    #     #     time.sleep(5)

    #     results_poi.update({"no_repump_saturation": 
    #                         {f"{power}":
    #                             {"scan_data": ple_data_logic.last_saved_files_paths,
    #                             "sigma": res["sigma"].value,
    #                             "sigma_stderr": res["sigma"].stderr,
    #                             "wavelength": wavelength
    #                             }
    #                             }})
    # if idx_no_ple_ohne_repump is None:
    #     continue
    # power_ = power_steps[idx_no_ple_checked : ][idx_no_ple_ohne_repump - 1]
    # pa.set_resonant_power(power = power_)
    # time.sleep(1)
    # pa.one_pulse_repump("blue")
    # time.sleep(0.5)
    # res = pa.do_ple_scan(lines = 20, in_range=fine_scan_range)
    # time.sleep(1)
    # #save_results
    # pa.save_ple(tag = f"lowest_power_{power_}",
    #     poi_name=poi_name, folder_name=folder_defect)

    # results_poi["lowest_power_scan"] = ple_data_logic.last_saved_files_paths


    # take spectrum to estimate SOC
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    ibeam_smart.enable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 100)
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    pa.take_spectrum()

    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(0.2)
    # spectrometer.fit_results.params["center_1"].value
    params = spectrometer.fit_results.params
    results_poi["SOC, GHz"] = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)

    ibeam_smart.disable()
    cobolt.disable_modulated()
    #cobolt.enable_modulated()
    #cobolt.set_laser_modulated_power(power = 5)

    with open(os.path.join(folder_defect, f'results_{poi_name}'), 'wb') as handle:
        pickle.dump(results_poi, handle, protocol=pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: 